基于FaceNet的人脸识别系统

**项目功能**：人脸注册、人脸识别、人脸验证

**技术栈**：PyTorch + MTCNN + Inception ResNet v1 + Triplet Loss

---
## 1. 环境配置

In [ ]:
# 安装依赖（首次运行取消注释）
# !pip install torch torchvision facenet-pytorch opencv-python matplotlib scikit-learn tqdm

In [ ]:
# ============================================================
# 环境配置与库导入
# ============================================================
# 设置离线模式，避免从Hugging Face Hub下载模型时网络超时
import os
os.environ['HF_HUB_OFFLINE'] = '1'          # 禁用HuggingFace在线下载
os.environ['TRANSFORMERS_OFFLINE'] = '1'    # 禁用Transformers在线下载

# ---- 基础库导入 ----
import random, pickle                       # random: 随机数生成; pickle: 对象序列化
import numpy as np                          # NumPy: 数值计算库
import matplotlib.pyplot as plt             # Matplotlib: 绑图库
from PIL import Image                       # PIL: 图像处理库
from tqdm import tqdm                       # tqdm: 进度条显示
from collections import defaultdict         # defaultdict: 带默认值的字典

# ---- PyTorch相关库导入 ----
import torch                                # PyTorch核心库
import torch.nn as nn                       # 神经网络模块
import torch.nn.functional as F             # 函数式API(激活函数、损失函数等)
import torch.optim as optim                 # 优化器(Adam、SGD等)
from torch.utils.data import Dataset, DataLoader  # 数据集和数据加载器
import torchvision.transforms as T          # 图像变换(数据增强)

# ---- 人脸识别专用库 ----
from facenet_pytorch import MTCNN, InceptionResnetV1  
# MTCNN: 多任务级联卷积网络，用于人脸检测和对齐
# InceptionResnetV1: FaceNet的骨干网络，用于特征提取

from sklearn.metrics import roc_curve, auc  # scikit-learn: ROC曲线和AUC计算

# ============================================================
# 配置Matplotlib中文字体支持
# ============================================================
import matplotlib
import platform

# 根据操作系统选择合适的中文字体
if platform.system() == 'Windows':
    # Windows系统优先使用微软雅黑
    plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
elif platform.system() == 'Darwin':  # macOS
    # macOS系统使用苹方或黑体
    plt.rcParams['font.sans-serif'] = ['PingFang SC', 'Heiti SC', 'Arial Unicode MS']
else:  # Linux
    # Linux系统使用文泉驿或Noto字体
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei', 'Noto Sans CJK SC', 'DejaVu Sans']

plt.rcParams['axes.unicode_minus'] = False  # 解决坐标轴负号'-'显示为方块的问题

# ============================================================
# 设置随机种子，确保实验可重复性
# ============================================================
random.seed(42)         # Python内置随机数种子
np.random.seed(42)      # NumPy随机数种子
torch.manual_seed(42)   # PyTorch CPU随机数种子
# 注: 如果使用GPU,还需设置 torch.cuda.manual_seed(42)

# ============================================================
# 选择计算设备(GPU/CPU)
# ============================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.cuda.is_available() 检查是否有可用的NVIDIA GPU
# 如果有GPU则使用'cuda'，否则使用'cpu'

print(f'PyTorch: {torch.__version__}, Device: {device}')
print(f'Matplotlib字体: {plt.rcParams["font.sans-serif"][:2]}')

---
## 2. 配置参数

In [ ]:
# ============================================================
# 项目配置类 - 集中管理所有超参数和路径
# ============================================================
class Config:
    """
    项目配置类
    将所有可调参数集中管理，便于实验调优和代码维护
    """
    
    # ---- 数据配置 ----
    DATA_ROOT = './lfw'              # LFW数据集根目录
    IMAGE_SIZE = 160                 # 输入图像尺寸，FaceNet标准为160×160
    MIN_IMAGES_PER_CLASS = 2         # 每个人最少需要的图片数(用于构建正样本对)
    
    # ---- 训练超参数 ----
    BATCH_SIZE = 64                  # 批次大小，影响GPU显存占用和训练稳定性
    NUM_WORKERS = 8                  # 数据加载的并行进程数，加速数据读取
    EPOCHS = 100                     # 训练轮数
    LEARNING_RATE = 0.0005           # 初始学习率，使用较小值避免破坏预训练权重
    WEIGHT_DECAY = 5e-4              # L2正则化系数，防止过拟合
    EMBEDDING_DIM = 128              # 人脸嵌入向量维度，FaceNet论文使用128
    PRETRAINED = 'vggface2'          # 预训练权重来源：vggface2或casia-webface
    MARGIN = 0.2                     # Triplet Loss的边界值，控制类间距离下限
    SAVE_FREQ = 10                   # 模型保存频率，每N轮保存一次
    THRESHOLD = 0.6                  # 人脸验证/识别的距离阈值
    
    # ---- 输出目录配置 ----
    OUTPUT_ROOT = './output'                      # 输出根目录
    CHECKPOINT_DIR = './output/checkpoints'       # 模型权重保存目录
    CACHE_DIR = './output/cache'                  # 数据缓存目录(预处理后的人脸)
    DATABASE_DIR = './output/database'            # 人脸数据库目录(注册的人脸特征)
    RESULTS_DIR = './output/results'              # 结果输出目录(图表、评估结果)
    
    # ---- 具体文件路径 ----
    CACHE_PATH = './output/cache/lfw_cache.pkl'           # 预处理人脸缓存文件
    DATABASE_PATH = './output/database/face_database.pkl' # 人脸数据库文件
    LOSS_CURVE_PATH = './output/results/loss_curve.png'   # 训练损失曲线图
    EVAL_RESULTS_PATH = './output/results/evaluation_results.png'  # 评估结果图
    MULTI_FACE_PATH = './output/results/multi_face_result.png'     # 多人脸识别结果图

# 创建配置实例
cfg = Config()

# 创建所有必要的输出目录
# os.makedirs(path, exist_ok=True) 会自动创建不存在的目录，已存在则跳过
for dir_path in [cfg.CHECKPOINT_DIR, cfg.CACHE_DIR, cfg.DATABASE_DIR, cfg.RESULTS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

# 打印关键配置信息
print('配置完成! ')
print(f'  BATCH_SIZE: {cfg.BATCH_SIZE}')
print(f'  EPOCHS: {cfg.EPOCHS}')
print(f'  LEARNING_RATE: {cfg.LEARNING_RATE}')

---
## 3. 数据集定义

In [ ]:
# ============================================================
# 数据增强策略定义
# ============================================================
# 数据增强是提高模型泛化能力的重要技术
# 通过对训练图像进行随机变换，让模型学习到更鲁棒的特征
# 项目要求：至少实现3种数据增强方法

train_transforms = T.Compose([
    # 1. 水平翻转 - 概率50%将图像左右镜像
    # 作用：人脸具有对称性，翻转后仍是有效样本，增加样本多样性
    T.RandomHorizontalFlip(p=0.5),
    
    # 2. 随机旋转 - 在±15度范围内随机旋转
    # 作用：模拟人脸在实际场景中的轻微倾斜
    T.RandomRotation(degrees=15),
    
    # 3. 颜色抖动 - 随机调整亮度、对比度、饱和度、色调
    # 作用：模拟不同光照条件，提高模型对光照变化的鲁棒性
    T.ColorJitter(
        brightness=0.3,    # 亮度变化范围：[1-0.3, 1+0.3]
        contrast=0.3,      # 对比度变化范围
        saturation=0.2,    # 饱和度变化范围
        hue=0.1           # 色调变化范围
    ),
    
    # 4. 随机仿射变换 - 随机平移
    # 作用：模拟人脸在图像中位置的微小变化
    T.RandomAffine(
        degrees=0,                      # 不额外旋转（已在上面处理）
        translate=(0.1, 0.1)           # 水平和垂直方向各最多平移10%
    ),
    
    # 5. 高斯模糊 - 随机添加模糊效果
    # 作用：模拟拍摄时的对焦模糊，提高模型对图像质量的容忍度
    T.GaussianBlur(
        kernel_size=3,                 # 模糊核大小
        sigma=(0.1, 2.0)              # 标准差范围，随机选择
    ),
])


# ============================================================
# LFW数据集类 - 带内存缓存版本
# ============================================================
class LFWDatasetCached(Dataset):
    """
    LFW (Labeled Faces in the Wild) 人脸数据集 - 内存缓存版本
    
    特点：
    1. 首次运行时使用MTCNN检测并对齐所有人脸，将结果缓存到磁盘
    2. 后续运行直接加载缓存，大幅提升数据加载速度
    3. 支持可选的数据增强
    
    LFW数据集简介：
    - 包含5749位公众人物的13233张人脸图像
    - 是人脸验证领域的标准测试基准
    """
    
    def __init__(self, root_dir, min_images=2, augment=False, cache_path=None):
        """
        初始化数据集
        
        Args:
            root_dir: LFW数据集根目录，结构为 root_dir/人名/图片.jpg
            min_images: 每个人物最少需要的图片数，少于此数的人物将被过滤
            augment: 是否启用数据增强（训练时为True，评估时为False）
            cache_path: 缓存文件路径
        """
        self.root_dir = root_dir
        self.augment = augment
        self.cache_path = cache_path or cfg.CACHE_PATH
        
        # 初始化MTCNN人脸检测器
        # image_size: 输出人脸图像尺寸
        # margin: 人脸边界框外扩像素数，保留更多上下文
        self.mtcnn = MTCNN(image_size=cfg.IMAGE_SIZE, margin=20, device=device)
        
        # 数据存储结构
        self.classes = []           # 类别名称列表（人名）
        self.class_to_idx = {}      # 类别名称到索引的映射
        self.samples = []           # 样本列表，每个元素为(face_tensor, label)
        
        # 尝试加载缓存，如果不存在则构建
        if os.path.exists(self.cache_path):
            print(f'加载缓存: {self.cache_path}')
            self._load_cache()
        else:
            print('构建数据集并缓存到内存...')
            self._build_and_cache(min_images)
    
    def _build_and_cache(self, min_images):
        """
        构建数据集并将所有预处理后的人脸缓存到磁盘
        
        流程：
        1. 遍历数据目录，筛选有足够图片的人物
        2. 对每张图片使用MTCNN检测人脸并对齐
        3. 将处理后的tensor保存到缓存文件
        """
        idx = 0                     # 类别索引计数器
        raw_samples = []            # 临时存储(图片路径, 标签)
        
        # 遍历数据目录
        for name in sorted(os.listdir(self.root_dir)):
            path = os.path.join(self.root_dir, name)
            if not os.path.isdir(path): 
                continue
            
            # 获取该人物的所有图片
            imgs = [f for f in os.listdir(path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
            
            # 过滤图片数量不足的人物
            if len(imgs) >= min_images:
                self.classes.append(name)
                self.class_to_idx[name] = idx
                for img in imgs:
                    raw_samples.append((os.path.join(path, img), idx))
                idx += 1
        
        print(f'类别: {len(self.classes)}, 图像: {len(raw_samples)}')
        
        # 预处理所有图像
        print('预处理人脸图像...')
        for path, label in tqdm(raw_samples, desc='Processing'):
            try:
                # 读取图像并转为RGB格式
                img = Image.open(path).convert('RGB')
                
                # 使用MTCNN检测并对齐人脸
                # 返回值为归一化后的人脸tensor，形状为[3, 160, 160]
                face = self.mtcnn(img)
                
                if face is None:
                    # 如果MTCNN检测失败，直接resize图像作为备选方案
                    img = img.resize((cfg.IMAGE_SIZE, cfg.IMAGE_SIZE))
                    face = T.ToTensor()(img)                    # 转为tensor [0,1]
                    face = T.Normalize([0.5]*3, [0.5]*3)(face)  # 归一化到[-1,1]
                
                # 保存到samples列表，注意转移到CPU以节省GPU显存
                self.samples.append((face.cpu(), label))
            except Exception as e:
                print(f'跳过 {path}: {e}')
        
        # 保存缓存到磁盘
        self._save_cache()
        print(f'缓存完成: {len(self.samples)} 张人脸')
    
    def _save_cache(self):
        """将数据集缓存保存到磁盘"""
        cache_data = {
            'classes': self.classes,
            'class_to_idx': self.class_to_idx,
            'samples': self.samples
        }
        with open(self.cache_path, 'wb') as f:
            pickle.dump(cache_data, f)
        print(f'缓存已保存: {self.cache_path}')
    
    def _load_cache(self):
        """从磁盘加载数据集缓存"""
        with open(self.cache_path, 'rb') as f:
            cache_data = pickle.load(f)
        self.classes = cache_data['classes']
        self.class_to_idx = cache_data['class_to_idx']
        self.samples = cache_data['samples']
        print(f'加载成功: {len(self.classes)} 类, {len(self.samples)} 张')
    
    def __len__(self): 
        """返回数据集大小"""
        return len(self.samples)
    
    def __getitem__(self, idx):
        """
        获取指定索引的样本
        
        Args:
            idx: 样本索引
        
        Returns:
            face: 人脸tensor，形状[3, 160, 160]
            label: 类别标签（整数）
        """
        face, label = self.samples[idx]
        
        # 训练时应用数据增强
        if self.augment:
            # 先将tensor转回PIL Image进行增强
            face_pil = T.ToPILImage()(face * 0.5 + 0.5)  # 反归一化：[-1,1] -> [0,1]
            face_pil = train_transforms(face_pil)         # 应用数据增强
            face = T.ToTensor()(face_pil)                 # 转回tensor
            face = T.Normalize([0.5]*3, [0.5]*3)(face)   # 重新归一化
        
        return face, label

print('数据增强策略: 水平翻转、随机旋转、颜色抖动、随机平移、高斯模糊')

In [ ]:
# ============================================================
# 三元组数据集类 - 用于Triplet Loss训练
# ============================================================
class TripletDataset(Dataset):
    """
    三元组数据集包装器
    
    将普通的(图像, 标签)数据集转换为(锚点, 正样本, 负样本, 标签)格式
    用于Triplet Loss训练
    
    三元组采样策略：
    - 锚点(Anchor): 当前样本
    - 正样本(Positive): 与锚点同一类别的另一个样本
    - 负样本(Negative): 与锚点不同类别的随机样本
    """
    
    def __init__(self, dataset):
        """
        Args:
            dataset: 基础数据集，需实现__getitem__返回(face, label)
        """
        self.dataset = dataset
        
        # 提取所有样本的标签
        self.labels = [s[1] for s in dataset.samples]
        
        # 构建标签到样本索引的映射表，便于快速查找同类样本
        # 例：{0: [0, 5, 10], 1: [1, 2, 8], ...}
        self.label_to_idx = defaultdict(list)
        for i, l in enumerate(self.labels):
            self.label_to_idx[l].append(i)
    
    def __len__(self): 
        return len(self.dataset)
    
    def __getitem__(self, idx):
        """
        获取一个三元组样本
        
        Returns:
            anchor: 锚点人脸
            positive: 正样本人脸（同一人的另一张照片）
            negative: 负样本人脸（不同人的照片）
            label: 锚点的类别标签
        """
        # 获取锚点样本
        anchor, label = self.dataset[idx]
        
        # 选择正样本：从同一类别中随机选择另一个样本
        pos_idx = idx
        if len(self.label_to_idx[label]) > 1:
            # 如果该类别有多个样本，则选择不同的样本
            while pos_idx == idx:
                pos_idx = random.choice(self.label_to_idx[label])
        positive, _ = self.dataset[pos_idx]
        
        # 选择负样本：从不同类别中随机选择
        neg_label = label
        while neg_label == label:
            neg_label = random.choice(list(self.label_to_idx.keys()))
        neg_idx = random.choice(self.label_to_idx[neg_label])
        negative, _ = self.dataset[neg_idx]
        
        return anchor, positive, negative, label

In [ ]:
# ============================================================
# 创建数据集和数据加载器
# ============================================================

print('加载数据集（内存缓存版）...')

# 创建基础数据集，启用数据增强（用于训练）
# 首次运行会处理所有图像并缓存，后续直接加载缓存
base_ds = LFWDatasetCached(
    cfg.DATA_ROOT,               # 数据集根目录
    cfg.MIN_IMAGES_PER_CLASS,    # 每人最少图片数
    augment=True                 # 启用数据增强
)

# 将基础数据集包装为三元组数据集
triplet_ds = TripletDataset(base_ds)

# 创建数据加载器
# DataLoader负责批量加载、打乱、多进程预取数据
train_loader = DataLoader(
    triplet_ds,
    batch_size=cfg.BATCH_SIZE,   # 每批样本数
    shuffle=True,                # 打乱数据顺序（训练时必须）
    num_workers=cfg.NUM_WORKERS  # 并行加载进程数
)

print(f'批次数: {len(train_loader)}')

---
## 4. 模型定义

In [ ]:
# ============================================================
# FaceNet模型定义
# ============================================================
class FaceNet(nn.Module):
    """
    FaceNet人脸识别模型
    
    架构：
    1. 骨干网络(Backbone): Inception ResNet v1
       - 输入: 160×160×3 RGB图像
       - 输出: 512维特征向量
       
    2. 嵌入层(Embedding): 线性层 + BatchNorm
       - 输入: 512维特征
       - 输出: 128维嵌入向量（L2归一化后）
    
    预训练权重：
    - VGGFace2: 330万张人脸图像训练，性能最佳
    - CASIA-WebFace: 50万张人脸图像训练
    """
    
    def __init__(self, pretrained='vggface2', emb_dim=128):
        """
        Args:
            pretrained: 预训练权重类型，'vggface2'或'casia-webface'或None
            emb_dim: 输出嵌入向量维度，默认128
        """
        super().__init__()
        
        # 创建骨干网络
        # pretrained=None 表示不自动下载权重，避免网络超时
        # classify=False 表示不添加分类头，只输出特征
        self.backbone = InceptionResnetV1(pretrained=None, classify=False)
        
        # 手动加载本地预训练权重
        if pretrained:
            # 权重文件标准缓存路径
            weight_path = os.path.expanduser('~/.cache/torch/hub/checkpoints/20180402-114759-vggface2.pt')
            
            if os.path.exists(weight_path):
                # 加载权重到CPU，避免GPU显存不足
                state_dict = torch.load(weight_path, map_location='cpu')
                
                # strict=False: 允许部分权重加载
                # 忽略不匹配的键（如分类层logits）
                self.backbone.load_state_dict(state_dict, strict=False)
                print(f'✓ 已加载本地权重: {weight_path}')
            else:
                print(f'✗ 权重文件不存在: {weight_path}')
                print('  请先下载权重到该路径，或设置 pretrained=None 使用随机初始化')
        
        # 嵌入层：将512维特征映射到目标维度
        self.embedding = nn.Sequential(
            nn.Linear(512, emb_dim),     # 全连接层
            nn.BatchNorm1d(emb_dim)      # 批归一化，稳定训练
        )
    
    def forward(self, x):
        """
        前向传播
        
        Args:
            x: 输入图像，形状[B, 3, 160, 160]
        
        Returns:
            嵌入向量，形状[B, 128]，已L2归一化
        """
        # 骨干网络提取特征
        feat = self.backbone(x)          # [B, 512]
        
        # 嵌入层降维
        emb = self.embedding(feat)       # [B, 128]
        
        # L2归一化：将向量投影到单位超球面上
        # 这使得欧氏距离等价于余弦相似度
        return F.normalize(emb, p=2, dim=1)

# 创建模型实例并移至计算设备
model = FaceNet(cfg.PRETRAINED, cfg.EMBEDDING_DIM).to(device)

# 打印模型参数量
print(f'模型参数: {sum(p.numel() for p in model.parameters()):,}')

In [ ]:
# ============================================================
# Triplet Loss 损失函数定义
# ============================================================
class TripletLoss(nn.Module):
    """
    基础三元组损失函数
    
    核心思想：
    让同一人的人脸特征距离（正样本距离）小于不同人的距离（负样本距离），
    且两者之间至少保持margin的间隔。
    
    损失公式：
    L = max(0, d(anchor, positive) - d(anchor, negative) + margin)
    
    其中：
    - d(a, p): 锚点与正样本的欧氏距离
    - d(a, n): 锚点与负样本的欧氏距离
    - margin: 边界值，控制类间最小距离
    
    训练目标：d(a, p) + margin < d(a, n)
    """
    
    def __init__(self, margin=0.2):
        """
        Args:
            margin: 边界值，典型值0.1~0.3
        """
        super().__init__()
        self.margin = margin
    
    def forward(self, anchor, positive, negative):
        """
        计算三元组损失
        
        Args:
            anchor: 锚点嵌入向量 [B, D]
            positive: 正样本嵌入向量 [B, D]
            negative: 负样本嵌入向量 [B, D]
        
        Returns:
            损失值（标量）
        """
        # 计算锚点-正样本距离
        pos_dist = F.pairwise_distance(anchor, positive)
        
        # 计算锚点-负样本距离
        neg_dist = F.pairwise_distance(anchor, negative)
        
        # Triplet Loss = max(0, pos_dist - neg_dist + margin)
        # F.relu 实现 max(0, x) 操作
        return F.relu(pos_dist - neg_dist + self.margin).mean()


# ============================================================
# 带硬负样本挖掘的Triplet Loss
# ============================================================
class TripletLossHardMining(nn.Module):
    """
    带硬负样本挖掘的三元组损失
    
    问题背景：
    随机采样的三元组中，大部分负样本距离锚点很远（简单负样本），
    这些样本的损失为0，对训练没有贡献。
    
    解决方案：
    在每个batch内，根据策略选择更有信息量的负样本。
    
    挖掘策略：
    1. random: 随机选择（基线）
    2. semi-hard: 选择满足 d(a,p) < d(a,n) < d(a,p)+margin 的负样本
       - 这类样本能提供有效梯度，同时训练稳定
    3. hard: 选择距离最近的负样本
       - 梯度最大，但可能导致训练不稳定
    """
    
    def __init__(self, margin=0.2, mining='semi-hard'):
        """
        Args:
            margin: 边界值
            mining: 挖掘策略，'random'/'semi-hard'/'hard'
        """
        super().__init__()
        self.margin = margin
        self.mining = mining
    
    def forward(self, embeddings, labels):
        """
        在batch内进行硬负样本挖掘并计算损失
        
        Args:
            embeddings: batch内所有样本的嵌入向量 [B, D]
            labels: 对应的标签 [B]
        
        Returns:
            损失值
        """
        # 计算batch内所有样本对的距离矩阵
        # dist_mat[i,j] = ||embeddings[i] - embeddings[j]||
        dist_mat = torch.cdist(embeddings, embeddings, p=2)
        batch_size = embeddings.size(0)
        
        # 构建同类/异类掩码
        labels = labels.view(-1, 1)
        same_id = (labels == labels.T).float()   # 同一人为1，不同人为0
        diff_id = 1 - same_id                     # 不同人为1
        
        # 正样本掩码：同类且不是自身
        mask_pos = same_id.clone()
        mask_pos.fill_diagonal_(0)
        
        if self.mining == 'hard':
            # 硬挖掘：选择同类中最远的正样本，异类中最近的负样本
            pos_dist = (dist_mat * mask_pos).max(dim=1)[0]
            neg_dist_mat = dist_mat + 1e6 * same_id  # 同类设为极大值
            neg_dist = neg_dist_mat.min(dim=1)[0]
            
        elif self.mining == 'semi-hard':
            # 半硬挖掘：正样本取平均，负样本取最近
            pos_dist = (dist_mat * mask_pos).sum(dim=1) / (mask_pos.sum(dim=1) + 1e-8)
            neg_dist_mat = dist_mat + 1e6 * same_id
            neg_dist = neg_dist_mat.min(dim=1)[0]
            
        else:  # random
            # 随机策略：取平均距离
            pos_dist = (dist_mat * mask_pos).sum(dim=1) / (mask_pos.sum(dim=1) + 1e-8)
            neg_dist = (dist_mat * diff_id).sum(dim=1) / (diff_id.sum(dim=1) + 1e-8)
        
        # 计算损失
        loss = F.relu(pos_dist - neg_dist + self.margin)
        return loss.mean()


# ============================================================
# 创建损失函数和优化器
# ============================================================

# 使用基础三元组损失（配合TripletDataset使用）
criterion = TripletLoss(cfg.MARGIN)

# 也可以使用硬挖掘版本（需要修改训练循环，传入embeddings和labels）
# criterion_hard = TripletLossHardMining(cfg.MARGIN, mining='semi-hard')

# Adam优化器
# - 自适应学习率，对每个参数使用不同的学习率
# - weight_decay: L2正则化，防止过拟合
optimizer = optim.Adam(
    model.parameters(), 
    lr=cfg.LEARNING_RATE, 
    weight_decay=cfg.WEIGHT_DECAY
)

# 余弦退火学习率调度器
# 学习率从初始值平滑衰减到eta_min，遵循余弦曲线
# 有助于模型收敛到更好的局部最优解
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=cfg.EPOCHS,      # 一个周期的步数
    eta_min=1e-6           # 最小学习率
)

print(f'损失函数: TripletLoss (margin={cfg.MARGIN})')
print(f'优化器: Adam (lr={cfg.LEARNING_RATE})')
print(f'学习率调度: CosineAnnealing (T_max={cfg.EPOCHS})')

---
## 5. 训练

In [ ]:
# ============================================================
# 训练函数定义
# ============================================================
def train_epoch(model, loader, criterion, optimizer):
    """
    执行一个epoch的训练
    
    训练流程：
    1. 遍历所有batch
    2. 前向传播：计算三元组的嵌入向量
    3. 计算损失：使用Triplet Loss
    4. 反向传播：计算梯度
    5. 参数更新：优化器更新权重
    
    Args:
        model: FaceNet模型
        loader: 数据加载器（返回anchor, positive, negative, label）
        criterion: 损失函数（TripletLoss）
        optimizer: 优化器（Adam）
    
    Returns:
        平均损失值
    """
    model.train()  # 设置为训练模式，启用Dropout和BatchNorm的训练行为
    total_loss, n = 0, 0
    
    # 遍历所有batch
    for a, p, neg, _ in tqdm(loader, desc='Training'):
        # 将数据移至计算设备（GPU/CPU）
        a = a.to(device)      # anchor: [B, 3, 160, 160]
        p = p.to(device)      # positive: [B, 3, 160, 160]
        neg = neg.to(device)  # negative: [B, 3, 160, 160]
        
        # 梯度清零
        # PyTorch默认会累积梯度，每个batch开始前需要清零
        optimizer.zero_grad()
        
        # 前向传播：计算三个样本的嵌入向量，然后计算损失
        anchor_emb = model(a)        # [B, 128]
        positive_emb = model(p)      # [B, 128]
        negative_emb = model(neg)    # [B, 128]
        loss = criterion(anchor_emb, positive_emb, negative_emb)
        
        # 反向传播：计算损失对每个参数的梯度
        loss.backward()
        
        # 参数更新：根据梯度更新模型权重
        optimizer.step()
        
        # 累积损失
        total_loss += loss.item()
        n += 1
    
    # 返回平均损失
    return total_loss / n

In [ ]:
# ============================================================
# 训练主循环
# ============================================================

# 训练历史记录
history = {'loss': []}

# 记录最佳损失，用于保存最优模型
best_loss = float('inf')

print('='*50)
print('开始训练')
print('='*50)

# 主训练循环
for epoch in range(cfg.EPOCHS):
    # 打印当前epoch信息和学习率
    current_lr = scheduler.get_last_lr()[0]
    print(f'\nEpoch {epoch+1}/{cfg.EPOCHS}, LR: {current_lr:.6f}')
    
    # 执行一个epoch的训练
    loss = train_epoch(model, train_loader, criterion, optimizer)
    
    # 记录损失历史
    history['loss'].append(loss)
    print(f'Loss: {loss:.4f}')
    
    # 更新学习率（余弦退火）
    scheduler.step()
    
    # 定期保存模型检查点
    if (epoch+1) % cfg.SAVE_FREQ == 0:
        checkpoint_path = f'{cfg.CHECKPOINT_DIR}/facenet_ep{epoch+1}.pth'
        torch.save(model.state_dict(), checkpoint_path)
        # 注：这里只保存了模型权重(state_dict)
        # 完整检查点应包含optimizer和scheduler状态，以便恢复训练
    
    # 保存最优模型（损失最低）
    if loss < best_loss:
        best_loss = loss
        torch.save(model.state_dict(), f'{cfg.CHECKPOINT_DIR}/facenet_best.pth')
        print('  -> 最优模型已保存')

print('\n训练完成!')
print(f'模型保存位置: {cfg.CHECKPOINT_DIR}/')

In [ ]:
# ============================================================
# 图像显示工具函数
# ============================================================
from IPython.display import HTML, display
import base64

def display_image(path, width=None):
    """
    以Base64嵌入方式显示图片
    
    解决Jupyter远程环境下图片无法显示的问题：
    - 远程服务器上plt.show()无法直接显示图片
    - 文件路径方式在某些环境下也有问题
    - Base64嵌入方式将图片编码到HTML中，兼容性最好
    
    Args:
        path: 图片文件路径
        width: 可选的显示宽度（像素）
    """
    # 读取图片文件并转为Base64编码
    with open(path, 'rb') as f:
        img_data = base64.b64encode(f.read()).decode()
    
    # 根据文件扩展名确定MIME类型
    ext = path.split('.')[-1].lower()
    mime = {
        'png': 'image/png', 
        'jpg': 'image/jpeg', 
        'jpeg': 'image/jpeg'
    }.get(ext, 'image/png')
    
    # 构造HTML img标签
    style = f'width:{width}px' if width else 'max-width:100%'
    html = f'<img src="data:{mime};base64,{img_data}" style="{style}"/>'
    
    # 在Notebook中显示
    display(HTML(html))

# ============================================================
# 绘制并保存训练损失曲线
# ============================================================
plt.figure(figsize=(10, 4))

# 绘制损失曲线
plt.plot(history['loss'], 'b-', lw=2)

# 设置图表标签
plt.xlabel('Epoch')       # X轴：训练轮次
plt.ylabel('Loss')        # Y轴：损失值
plt.title('Training Loss')  # 标题
plt.grid(True)            # 显示网格

# 保存图表到文件
plt.savefig(cfg.LOSS_CURVE_PATH, dpi=150)
plt.close()  # 关闭图形，释放内存

# 显示保存的图片
print(f'训练曲线已保存: {cfg.LOSS_CURVE_PATH}')
display_image(cfg.LOSS_CURVE_PATH)

---
## 6. 人脸注册

In [ ]:
# ============================================================
# 人脸数据库类 - 用于人脸注册功能
# ============================================================
class FaceDatabase:
    """
    人脸数据库管理类
    
    功能：
    1. 注册人脸：将人脸图像转换为嵌入向量并存储
    2. 持久化：支持将数据库保存到文件和从文件加载
    
    数据结构：
    db = {
        '张三': numpy array [N, 128],  # N张照片的嵌入向量
        '李四': numpy array [M, 128],
        ...
    }
    
    使用流程：
    1. 创建数据库：face_db = FaceDatabase(model)
    2. 注册人脸：face_db.register('姓名', ['图片1.jpg', '图片2.jpg'])
    3. 保存数据库：face_db.save('database.pkl')
    4. 加载数据库：face_db.load('database.pkl')
    """
    
    def __init__(self, model):
        """
        Args:
            model: FaceNet模型，用于提取人脸特征
        """
        self.model = model
        
        # 初始化MTCNN人脸检测器
        self.mtcnn = MTCNN(image_size=cfg.IMAGE_SIZE, margin=20, device=device)
        
        # 人脸数据库字典：{姓名: 嵌入向量数组}
        self.db = {}
    
    def register(self, name, paths):
        """
        注册人脸到数据库
        
        Args:
            name: 人员姓名/ID
            paths: 该人员的人脸图片路径列表
        
        Returns:
            True: 注册成功
            False: 注册失败（无有效人脸）
        """
        self.model.eval()  # 设置为评估模式
        embs = []
        
        for p in paths:
            # 读取图像
            img = Image.open(p).convert('RGB')
            
            # 检测并对齐人脸
            face = self.mtcnn(img)
            
            if face is not None:
                # 提取嵌入向量
                with torch.no_grad():  # 禁用梯度计算，节省内存
                    emb = self.model(face.unsqueeze(0).to(device))
                embs.append(emb.cpu().numpy())
        
        if embs:
            # 将多张照片的嵌入向量堆叠为数组
            self.db[name] = np.vstack(embs)
            print(f'注册成功: {name} ({len(embs)}张)')
            return True
        return False
    
    def register_folder(self, folder):
        """
        从文件夹批量注册人脸
        
        Args:
            folder: 包含人脸图片的文件夹路径
                   文件夹名将作为人员姓名
        
        Returns:
            注册是否成功
        """
        # 使用文件夹名作为人员姓名
        name = os.path.basename(folder)
        
        # 获取文件夹中的所有图片
        paths = [
            os.path.join(folder, f) 
            for f in os.listdir(folder) 
            if f.endswith(('.jpg', '.png'))
        ]
        
        return self.register(name, paths)
    
    def save(self, path):
        """
        保存数据库到文件
        
        Args:
            path: 保存路径（.pkl文件）
        """
        with open(path, 'wb') as f:
            pickle.dump(self.db, f)
        print(f'数据库保存: {path}')
    
    def load(self, path):
        """
        从文件加载数据库
        
        Args:
            path: 数据库文件路径
        """
        with open(path, 'rb') as f:
            self.db = pickle.load(f)
        print(f'数据库加载: {len(self.db)}人')

In [ ]:
# ============================================================
# 人脸注册示例
# ============================================================

# 创建人脸数据库实例
face_db = FaceDatabase(model)

# 从LFW数据集中选择前5个人物进行注册
# 获取所有人物目录
sample_persons = [
    d for d in os.listdir(cfg.DATA_ROOT) 
    if os.path.isdir(os.path.join(cfg.DATA_ROOT, d))
][:5]  # 取前5个

# 逐个注册
for person in sample_persons:
    person_folder = os.path.join(cfg.DATA_ROOT, person)
    face_db.register_folder(person_folder)

# 保存数据库到文件
face_db.save(cfg.DATABASE_PATH)
print(f'人脸数据库已保存: {cfg.DATABASE_PATH}')

---
## 7. 人脸识别

In [ ]:
# ============================================================
# 人脸识别类 - 1:N搜索
# ============================================================
class FaceRecognizer:
    """
    人脸识别器（1:N搜索）
    
    功能：
    给定一张人脸图片，在数据库中搜索最相似的人物
    
    识别流程：
    1. 检测并对齐输入图像中的人脸
    2. 提取人脸的嵌入向量
    3. 计算与数据库中所有人脸的距离
    4. 返回距离最近的人物（如果距离小于阈值）
    
    应用场景：
    - 门禁系统：识别来访人员身份
    - 考勤系统：自动打卡
    - 照片管理：按人物分类照片
    """
    
    def __init__(self, model, database, threshold=0.6):
        """
        Args:
            model: FaceNet模型
            database: FaceDatabase实例
            threshold: 识别阈值，距离小于此值才认为是已知人物
        """
        self.model = model
        self.db = database
        self.threshold = threshold
        self.mtcnn = MTCNN(image_size=cfg.IMAGE_SIZE, margin=20, device=device)
    
    def recognize(self, image_path):
        """
        识别图像中的人脸
        
        Args:
            image_path: 待识别图片路径
        
        Returns:
            name: 识别到的人物姓名（或'Unknown'/'No face'）
            confidence: 置信度（0~1，越高越确信）
            distance: 与最近人物的距离
        """
        self.model.eval()
        
        # 读取图像
        img = Image.open(image_path).convert('RGB')
        
        # 检测人脸
        face = self.mtcnn(img)
        if face is None:
            return 'No face', 0, float('inf')
        
        # 提取嵌入向量
        with torch.no_grad():
            emb = self.model(face.unsqueeze(0).to(device)).cpu().numpy()
        
        # 在数据库中搜索最相似的人物
        min_dist = float('inf')
        best = 'Unknown'
        
        for name, db_embs in self.db.db.items():
            # 计算与该人物所有注册照片的距离，取最小值
            # 这样即使某张注册照片质量不好，也不会影响识别
            d = np.linalg.norm(db_embs - emb, axis=1).min()
            if d < min_dist:
                min_dist, best = d, name
        
        # 计算置信度：距离越小，置信度越高
        # 使用线性映射：conf = 1 - dist/2，确保结果在[0,1]范围内
        conf = max(0, 1 - min_dist/2)
        
        # 距离小于阈值才返回识别结果，否则返回Unknown
        if min_dist < self.threshold:
            return best, conf, min_dist
        else:
            return 'Unknown', conf, min_dist
    
    def recognize_show(self, path, save_path=None):
        """
        识别并可视化结果
        
        Args:
            path: 待识别图片路径
            save_path: 结果图片保存路径
        
        Returns:
            name: 识别结果
            conf: 置信度
        """
        # 执行识别
        name, conf, dist = self.recognize(path)
        save_path = save_path or f'{cfg.RESULTS_DIR}/recognize_result.png'
        
        # 绘制结果图
        plt.figure(figsize=(6, 6))
        plt.imshow(Image.open(path))
        plt.title(f'{name} (置信度:{conf:.1%}, 距离:{dist:.3f})')
        plt.axis('off')
        
        # 保存并显示
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        display_image(save_path)
        
        return name, conf

In [ ]:
# ============================================================
# 人脸识别功能测试
# ============================================================

# 创建识别器实例
recognizer = FaceRecognizer(model, face_db, cfg.THRESHOLD)

# 选择测试图片：使用已注册人员的一张照片
test_person = sample_persons[0]  # 取第一个已注册的人
test_folder = os.path.join(cfg.DATA_ROOT, test_person)
test_img = os.path.join(test_folder, os.listdir(test_folder)[0])

# 执行识别并显示结果
print(f'测试图像: {test_img}')
recognizer.recognize_show(test_img, save_path=f'{cfg.RESULTS_DIR}/recognize_demo.png')

---
## 8. 人脸验证

In [ ]:
# ============================================================
# 人脸验证类 - 1:1比对
# ============================================================
class FaceVerifier:
    """
    人脸验证器（1:1比对）
    
    功能：
    判断两张照片是否为同一人
    
    验证流程：
    1. 分别检测并对齐两张图片中的人脸
    2. 提取两张人脸的嵌入向量
    3. 计算两个向量之间的欧氏距离
    4. 距离小于阈值则判定为同一人
    
    与识别的区别：
    - 验证（1:1）：已知声称的身份，判断是否属实
    - 识别（1:N）：不知道身份，在数据库中搜索
    
    应用场景：
    - 身份认证：解锁手机、登录账户
    - 证件核验：人证比对
    - 双胞胎区分
    """
    
    def __init__(self, model, threshold=0.6):
        """
        Args:
            model: FaceNet模型
            threshold: 验证阈值，距离小于此值判定为同一人
        """
        self.model = model
        self.threshold = threshold
        self.mtcnn = MTCNN(image_size=cfg.IMAGE_SIZE, margin=20, device=device)
    
    def verify(self, path1, path2):
        """
        验证两张照片是否为同一人
        
        Args:
            path1: 第一张图片路径
            path2: 第二张图片路径
        
        Returns:
            is_same: 是否为同一人（True/False/None）
            distance: 两个嵌入向量之间的距离
            confidence: 置信度
        """
        self.model.eval()
        faces = []
        
        # 检测两张图片中的人脸
        for p in [path1, path2]:
            img = Image.open(p).convert('RGB')
            f = self.mtcnn(img)
            if f is None:
                return None, None, 'Face not detected'
            faces.append(f)
        
        # 提取嵌入向量
        with torch.no_grad():
            e1 = self.model(faces[0].unsqueeze(0).to(device))
            e2 = self.model(faces[1].unsqueeze(0).to(device))
        
        # 计算欧氏距离
        dist = F.pairwise_distance(e1, e2).item()
        
        # 判定结果
        is_same = dist < self.threshold
        
        # 计算置信度
        conf = max(0, 1 - dist/2)
        
        return is_same, dist, conf
    
    def verify_show(self, p1, p2, save_path=None):
        """
        验证并可视化结果
        
        Args:
            p1: 第一张图片路径
            p2: 第二张图片路径
            save_path: 结果保存路径
        
        Returns:
            验证结果（True/False/None）
        """
        # 执行验证
        result, dist, conf = self.verify(p1, p2)
        save_path = save_path or f'{cfg.RESULTS_DIR}/verify_result.png'
        
        # 绘制对比图
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        
        # 显示两张图片
        ax[0].imshow(Image.open(p1))
        ax[0].set_title('Image 1')
        ax[0].axis('off')
        
        ax[1].imshow(Image.open(p2))
        ax[1].set_title('Image 2')
        ax[1].axis('off')
        
        # 在图片上方显示验证结果
        status = '同一人 ✓' if result else '不同人 ✗'
        plt.suptitle(f'{status} | 距离:{dist:.3f} | 置信度:{conf:.1%}')
        
        # 保存并显示
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        display_image(save_path)
        
        return result

In [ ]:
# ============================================================
# 人脸验证功能测试
# ============================================================

# 创建验证器实例
verifier = FaceVerifier(model, cfg.THRESHOLD)

# 寻找有至少2张图片的人物进行验证测试
# （验证需要同一人的两张不同照片）
test_person = None
test_folder = None

# 首先在已注册的人物中查找
for person in sample_persons:
    folder = os.path.join(cfg.DATA_ROOT, person)
    imgs = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if len(imgs) >= 2:
        test_person = person
        test_folder = folder
        break

# 执行验证测试
if test_folder and test_person:
    # 获取该人物的前两张照片
    imgs = os.listdir(test_folder)[:2]
    p1 = os.path.join(test_folder, imgs[0])
    p2 = os.path.join(test_folder, imgs[1])
    
    print(f'验证测试人物: {test_person}')
    print(f'图片1: {imgs[0]}')
    print(f'图片2: {imgs[1]}')
    
    # 执行验证并显示结果
    verifier.verify_show(p1, p2, save_path=f'{cfg.RESULTS_DIR}/verify_demo.png')
else:
    # 如果已注册人物都只有1张图片，从整个数据集中查找
    for person in os.listdir(cfg.DATA_ROOT):
        folder = os.path.join(cfg.DATA_ROOT, person)
        if os.path.isdir(folder):
            imgs = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if len(imgs) >= 2:
                p1 = os.path.join(folder, imgs[0])
                p2 = os.path.join(folder, imgs[1])
                print(f'验证测试人物: {person}')
                verifier.verify_show(p1, p2, save_path=f'{cfg.RESULTS_DIR}/verify_demo.png')
                break
    else:
        print('数据集中没有找到有2张以上图片的人物')

---
## 9. 模型评估

使用VGGFace2预训练模型评估LFW准确率。

In [ ]:
# ============================================================
# 模型评估模块
# ============================================================
import time

# 创建评估数据集（不启用数据增强）
# 评估时应使用原始图像，避免增强带来的随机性
print('创建无增强评估数据集...')
eval_ds = LFWDatasetCached(cfg.DATA_ROOT, cfg.MIN_IMAGES_PER_CLASS, augment=False)

print('\n' + '='*60)
print('模型评估：VGGFace2预训练模型')
print('='*60)

# ============================================================
# 加载VGGFace2预训练模型（原始512维嵌入）
# ============================================================
weight_path = os.path.expanduser('~/.cache/torch/hub/checkpoints/20180402-114759-vggface2.pt')

if os.path.exists(weight_path):
    # 创建原始InceptionResnetV1模型（不经过额外的嵌入层）
    pretrained_model = InceptionResnetV1(pretrained=None, classify=False).to(device)
    
    # 加载预训练权重
    state_dict = torch.load(weight_path, map_location=device)
    pretrained_model.load_state_dict(state_dict, strict=False)
    pretrained_model.eval()
    
    print(f'✓ 已加载VGGFace2预训练权重')
    print(f'  输出维度: 512维嵌入向量')
else:
    print(f'✗ 权重文件不存在: {weight_path}')
    pretrained_model = None


# ============================================================
# LFW评估函数
# ============================================================
def evaluate_model_lfw(model, dataset, n_pairs=2000, save_path=None):
    """
    在LFW数据集上评估模型性能
    
    评估方法：
    1. 随机生成n_pairs对正样本对（同一人的两张照片）
    2. 随机生成n_pairs对负样本对（不同人的照片）
    3. 计算所有样本对的距离
    4. 搜索最优阈值（使准确率最高的阈值）
    5. 计算ROC曲线和AUC
    
    Args:
        model: 要评估的模型
        dataset: 评估数据集
        n_pairs: 生成的样本对数量
        save_path: 结果图保存路径
    
    Returns:
        accuracy: 最优阈值下的准确率
        threshold: 最优阈值
        auc: AUC分数
    """
    if model is None:
        print('错误：模型未加载')
        return 0, 0, 0
    
    model.eval()
    save_path = save_path or cfg.EVAL_RESULTS_PATH
    
    labels, dists = [], []  # 标签和距离列表
    samples = dataset.samples
    
    # 构建标签到索引的映射
    label_to_idx = defaultdict(list)
    for i, sample in enumerate(samples):
        label_to_idx[sample[1]].append(i)
    
    print('生成评估对...')
    
    for _ in tqdm(range(n_pairs), desc='Evaluating'):
        # 筛选有至少2张图片的类别（用于构建正样本对）
        valid_labels = [l for l in label_to_idx.keys() if len(label_to_idx[l]) >= 2]
        if not valid_labels:
            continue
        
        # 随机选择一个类别
        label = random.choice(valid_labels)
        
        # 从该类别中随机选择2个样本作为正样本对
        i1, i2 = random.sample(label_to_idx[label], 2)
        
        try:
            # 获取两张人脸的tensor
            f1 = samples[i1][0].unsqueeze(0).to(device)
            f2 = samples[i2][0].unsqueeze(0).to(device)
            
            # 提取嵌入向量并计算距离
            with torch.no_grad():
                e1, e2 = model(f1), model(f2)
            dists.append(F.pairwise_distance(e1, e2).item())
            labels.append(1)  # 正样本标签为1
        except:
            continue
        
        # 生成负样本对（不同人）
        l2 = label
        while l2 == label:
            l2 = random.choice(list(label_to_idx.keys()))
        i3 = random.choice(label_to_idx[l2])
        
        try:
            f3 = samples[i3][0].unsqueeze(0).to(device)
            with torch.no_grad():
                e3 = model(f3)
            dists.append(F.pairwise_distance(e1, e3).item())
            labels.append(0)  # 负样本标签为0
        except:
            continue
    
    # 转换为numpy数组
    dists, labels = np.array(dists), np.array(labels)
    print(f'收集到 {len(dists)} 个评估对 (正样本:{sum(labels)}, 负样本:{len(labels)-sum(labels)})')
    
    # ============================================================
    # 搜索最优阈值
    # ============================================================
    best_acc, best_th = 0, 0
    for th in np.arange(0.1, 2.0, 0.01):
        # 预测：距离小于阈值则预测为同一人(1)
        preds = (dists < th).astype(int)
        acc = (preds == labels).mean()
        if acc > best_acc:
            best_acc, best_th = acc, th
    
    # ============================================================
    # 计算ROC曲线
    # ============================================================
    # 注意：这里使用-dists是因为距离越小越可能是同一人
    # sklearn的roc_curve期望分数越高越可能是正类
    fpr, tpr, _ = roc_curve(labels, -dists)
    roc_auc = auc(fpr, tpr)
    
    # ============================================================
    # 可视化结果
    # ============================================================
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图：ROC曲线
    axes[0].plot(fpr, tpr, 'b-', lw=2, label=f'ROC (AUC={roc_auc:.3f})')
    axes[0].plot([0, 1], [0, 1], 'r--', lw=1)  # 对角线（随机猜测基准）
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title('ROC Curve')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 右图：距离分布直方图
    pos_dists, neg_dists = dists[labels == 1], dists[labels == 0]
    axes[1].hist(pos_dists, bins=30, alpha=0.6, label='Same Person', color='green')
    axes[1].hist(neg_dists, bins=30, alpha=0.6, label='Different Person', color='red')
    axes[1].axvline(x=best_th, color='blue', linestyle='--', label=f'Threshold={best_th:.2f}')
    axes[1].set_xlabel('Distance')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Distance Distribution')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()
    
    display_image(save_path)
    
    return best_acc, best_th, roc_auc


# ============================================================
# 效率测试函数
# ============================================================
def measure_efficiency(model, dataset, n_tests=50):
    """
    测量特征提取和数据库搜索的时间效率
    
    Args:
        model: 模型
        dataset: 数据集
        n_tests: 测试次数
    
    Returns:
        extract_time: 单张图片特征提取时间（毫秒）
        search_time: 数据库搜索时间（毫秒）
    """
    model.eval()
    samples = dataset.samples
    
    # 测量特征提取时间
    extract_times = []
    for i in range(min(20, len(samples))):
        face = samples[i][0].unsqueeze(0).to(device)
        
        start = time.time()
        with torch.no_grad():
            _ = model(face)
        
        # GPU需要同步才能准确计时
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        extract_times.append(time.time() - start)
    
    # 模拟数据库搜索：在100人的数据库中查找最近邻
    db_size = 100
    db_embs = torch.randn(db_size, 512).to(device)
    query_emb = torch.randn(1, 512).to(device)
    
    search_times = []
    for _ in range(20):
        start = time.time()
        dists = torch.cdist(query_emb, db_embs)
        _ = dists.argmin().item()
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        search_times.append(time.time() - start)
    
    # 转换为毫秒
    extract_time = np.mean(extract_times) * 1000
    search_time = np.mean(search_times) * 1000
    
    return extract_time, search_time


# ============================================================
# 执行评估
# ============================================================
if pretrained_model is not None:
    print('\n运行模型评估...')
    acc, threshold, auc_score = evaluate_model_lfw(
        pretrained_model, eval_ds, n_pairs=2000, 
        save_path=cfg.EVAL_RESULTS_PATH
    )
    
    print('\n运行效率测试...')
    extract_time, search_time = measure_efficiency(pretrained_model, eval_ds)
    
    # 打印评估结果摘要
    print(f'\n{"="*60}')
    print(f'{"模型评估结果":^56}')
    print(f'{"="*60}')
    print(f'LFW准确率: {acc:.2%} (目标: 97%+) {"✓" if acc >= 0.97 else ""}')
    print(f'AUC分数: {auc_score:.3f}')
    print(f'最优阈值: {threshold:.3f}')
    print(f'特征提取: {extract_time:.1f} ms/张')
    print(f'数据库搜索(100人): {search_time:.2f} ms')
    print(f'{"="*60}')
else:
    print('跳过评估：模型未加载')

---
## 10.项目总结

### 项目完成度检查

| 要求项 | 状态 | 实现说明 |
|--------|------|----------|
| **数据准备** | ✓ | LFW数据集 |
| 人脸检测与对齐 | ✓ | MTCNN |
| 图像尺寸160×160 | ✓ | Config配置 |
| 数据增强(≥3种) | ✓ | 翻转/旋转/颜色抖动/平移/模糊 |
| **模型实现** | ✓ | - |
| Inception ResNet v1 | ✓ | facenet-pytorch |
| 128维嵌入向量 | ✓ | embedding层 |
| 三元组损失 | ✓ | TripletLoss |
| 硬负样本挖掘 | ✓ | TripletLossHardMining |
| 学习率调度 | ✓ | CosineAnnealing |
| 预训练微调 | ✓ | vggface2权重 |
| **系统功能** | ✓ | - |
| 人脸注册 | ✓ | FaceDatabase |
| 人脸识别 | ✓ | FaceRecognizer |
| 人脸验证 | ✓ | FaceVerifier |
| **评估分析** | ✓ | - |
| LFW准确率 | ✓ | evaluate_model |
| 时间效率 | ✓ | measure_efficiency |
| ROC曲线 | ✓ | 可视化输出 |
| **加分项** | ✓ | - |
| 多人脸检测与识别 | ✓ | MultiFaceRecognizer |

### 技术栈
- **深度学习框架**: PyTorch
- **人脸检测**: MTCNN (支持多人脸)
- **骨干网络**: Inception ResNet v1
- **损失函数**: Triplet Loss (支持硬负样本挖掘)
- **嵌入维度**: 128维

### 文件结构
```
facenet/
├── facenet_project.ipynb       # 主项目Notebook
├── requirements.txt            # 依赖文件
├── lfw/                        # LFW数据集
└── output/                     # 所有输出文件
    ├── checkpoints/            # 模型检查点
    │   ├── facenet_best.pth    # 最优模型
    │   └── facenet_ep*.pth     # 各轮次模型
    ├── cache/                  # 数据缓存
    │   └── lfw_cache.pkl       # 预处理人脸缓存
    ├── database/               # 人脸数据库
    │   └── face_database.pkl   # 注册人员特征
    └── results/                # 评估结果
        ├── loss_curve.png      # 训练损失曲线
        ├── evaluation_results.png  # ROC曲线等
        ├── multi_face_result.png   # 多人脸识别结果
        └── multi_face_test.jpg     # 多人脸测试图
```

In [ ]:
# ============================================================
# 多人脸检测与识别类（加分项）
# ============================================================
import cv2

class MultiFaceRecognizer:
    """
    多人脸检测与识别器
    
    功能：
    在单张图像中检测并识别多个人脸
    
    与FaceRecognizer的区别：
    - FaceRecognizer: 假设图像中只有一张人脸
    - MultiFaceRecognizer: 支持检测和识别多张人脸
    
    实现关键：
    使用MTCNN的keep_all=True选项，保留检测到的所有人脸
    
    应用场景：
    - 合影照片的人物标注
    - 监控视频中的多人追踪
    - 会议签到系统
    """
    
    def __init__(self, model, database, threshold=0.6):
        """
        Args:
            model: FaceNet模型
            database: FaceDatabase实例
            threshold: 识别阈值
        """
        self.model = model
        self.db = database
        self.threshold = threshold
        
        # 创建MTCNN检测器，启用多人脸检测模式
        self.mtcnn = MTCNN(
            image_size=cfg.IMAGE_SIZE,
            margin=20,
            keep_all=True,  # 关键参数：保留所有检测到的人脸
            device=device
        )
    
    def detect_faces(self, image):
        """
        检测图像中的所有人脸
        
        Args:
            image: PIL Image 或 numpy数组（BGR格式）
        
        Returns:
            faces: 人脸tensor列表，形状[N, 3, 160, 160]
            boxes: 边界框数组，形状[N, 4]，格式为[x1, y1, x2, y2]
            probs: 检测概率数组，形状[N]
        """
        # 如果输入是OpenCV格式（BGR numpy数组），转换为PIL Image
        if isinstance(image, np.ndarray):
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # 检测人脸边界框和概率
        boxes, probs = self.mtcnn.detect(image)
        
        if boxes is None:
            return None, None, None
        
        # 提取对齐后的人脸图像
        faces = self.mtcnn(image)
        
        return faces, boxes, probs
    
    def recognize_all(self, image_path):
        """
        检测并识别图像中的所有人脸
        
        Args:
            image_path: 图像文件路径
        
        Returns:
            results: 结果列表，每个元素为字典：
                {
                    'name': 人物姓名或'Unknown',
                    'confidence': 置信度,
                    'distance': 与最近人物的距离,
                    'box': 边界框[x1, y1, x2, y2],
                    'detection_prob': 人脸检测概率
                }
        """
        self.model.eval()
        
        # 读取图像
        img = Image.open(image_path).convert('RGB')
        
        # 检测所有人脸
        faces, boxes, probs = self.detect_faces(img)
        
        if faces is None:
            return []
        
        results = []
        
        # 将所有人脸移至GPU并批量提取特征
        faces = faces.to(device)
        with torch.no_grad():
            embeddings = self.model(faces)
        embeddings = embeddings.cpu().numpy()
        
        # 对每张人脸进行识别
        for i, (emb, box, prob) in enumerate(zip(embeddings, boxes, probs)):
            min_dist, best_name = float('inf'), 'Unknown'
            
            # 在数据库中搜索最相似的人物
            for name, db_embs in self.db.db.items():
                dists = np.linalg.norm(db_embs - emb, axis=1)
                d = dists.min()
                if d < min_dist:
                    min_dist, best_name = d, name
            
            # 距离超过阈值则标记为Unknown
            if min_dist > self.threshold:
                best_name = 'Unknown'
            
            # 计算置信度
            confidence = max(0, 1 - min_dist / 2)
            
            results.append({
                'name': best_name,
                'confidence': confidence,
                'distance': min_dist,
                'box': box.astype(int),
                'detection_prob': prob
            })
        
        return results
    
    def visualize(self, image_path, save_path=None):
        """
        检测识别并可视化结果
        
        在图像上绘制边界框和识别结果
        
        Args:
            image_path: 输入图像路径
            save_path: 输出图像保存路径
        
        Returns:
            results: 识别结果列表
        """
        # 使用OpenCV读取图像（用于绘制）
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 执行识别
        results = self.recognize_all(image_path)
        
        if not results:
            print('未检测到人脸')
            plt.figure(figsize=(12, 8))
            plt.imshow(img_rgb)
            plt.title('No faces detected')
            plt.axis('off')
            if save_path:
                plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
            if save_path:
                display_image(save_path)
            return results
        
        # 为每个人脸分配不同颜色
        colors = plt.cm.Set1(np.linspace(0, 1, 10))[:, :3] * 255
        
        # 在图像上绘制边界框和标签
        for i, res in enumerate(results):
            box = res['box']
            name = res['name']
            conf = res['confidence']
            color = tuple(map(int, colors[i % len(colors)]))
            
            # 绘制边界框
            cv2.rectangle(img_rgb, (box[0], box[1]), (box[2], box[3]), color, 2)
            
            # 绘制标签背景和文字
            label = f"{name} ({conf:.1%})"
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(img_rgb, (box[0], box[1]-25), (box[0]+w+5, box[1]), color, -1)
            cv2.putText(img_rgb, label, (box[0]+2, box[1]-8),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # 使用matplotlib显示结果
        plt.figure(figsize=(12, 8))
        plt.imshow(img_rgb)
        plt.title(f'Multi-Face Recognition: {len(results)} faces detected')
        plt.axis('off')
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
            print(f'结果已保存: {save_path}')
            display_image(save_path)
        else:
            plt.close()
        
        # 打印识别结果摘要
        print(f'\n检测到 {len(results)} 张人脸:')
        print('-' * 50)
        for i, res in enumerate(results, 1):
            print(f"{i}. {res['name']}: 置信度={res['confidence']:.1%}, "
                  f"距离={res['distance']:.3f}, 检测概率={res['detection_prob']:.1%}")
        
        return results


print('MultiFaceRecognizer 类定义完成！')

In [ ]:
# ============================================================
# 多人脸识别功能测试
# ============================================================

# 创建多人脸识别器实例
multi_recognizer = MultiFaceRecognizer(model, face_db, cfg.THRESHOLD)

# 从LFW获取测试图片
# 注意：由于使用了缓存版数据集，samples中存储的是tensor而非路径
# 因此需要从原始目录获取图片路径
test_person = [
    d for d in os.listdir(cfg.DATA_ROOT) 
    if os.path.isdir(os.path.join(cfg.DATA_ROOT, d))
][0]  # 取第一个人物

test_folder = os.path.join(cfg.DATA_ROOT, test_person)
test_img_path = os.path.join(test_folder, os.listdir(test_folder)[0])

print(f'测试图像: {test_img_path}')

# 运行多人脸识别并保存结果
results = multi_recognizer.visualize(test_img_path, save_path=cfg.MULTI_FACE_PATH)
print(f'结果已保存: {cfg.MULTI_FACE_PATH}')

In [ ]:
# ============================================================
# 创建多人脸测试图像
# ============================================================
def create_multi_face_image(registered_persons, n_unknown=2, save_path=None):
    """
    创建包含已注册和未注册人脸的测试图像
    
    通过拼接多个单人照片来模拟合影场景
    用于测试多人脸识别功能
    
    Args:
        registered_persons: 已注册人员名单
        n_unknown: 未注册人员数量（用于测试Unknown识别）
        save_path: 保存路径
    
    Returns:
        保存的图像路径
    """
    save_path = save_path or f'{cfg.RESULTS_DIR}/multi_face_test.jpg'
    
    # 获取所有人物目录
    all_persons = [
        d for d in os.listdir(cfg.DATA_ROOT) 
        if os.path.isdir(os.path.join(cfg.DATA_ROOT, d))
    ]
    
    # 选择2个已注册的人（测试正确识别）
    registered_in_db = [p for p in registered_persons if p in all_persons][:2]
    
    # 选择n_unknown个未注册的人（测试Unknown识别）
    unregistered = [p for p in all_persons if p not in registered_persons]
    unknown_persons = random.sample(unregistered, min(n_unknown, len(unregistered)))
    
    selected_persons = registered_in_db + unknown_persons
    
    print(f'已注册人员: {registered_in_db}')
    print(f'未注册人员: {unknown_persons}')
    
    # 收集选中人物的照片
    images = []
    for person in selected_persons:
        person_folder = os.path.join(cfg.DATA_ROOT, person)
        img_files = [
            f for f in os.listdir(person_folder) 
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
        if img_files:
            img_path = os.path.join(person_folder, img_files[0])
            img = Image.open(img_path).convert('RGB')
            img = img.resize((200, 200))  # 统一尺寸
            images.append(np.array(img))
    
    if not images:
        print('错误：没有找到测试图片')
        return None
    
    # 创建拼接画布（2行N列）
    rows = 2
    cols = (len(images) + 1) // 2
    h, w = 200, 200
    canvas = np.ones((rows * h, cols * w, 3), dtype=np.uint8) * 255  # 白色背景
    
    # 将图片放置到画布上
    for i, img in enumerate(images):
        r, c = i // cols, i % cols
        canvas[r*h:(r+1)*h, c*w:(c+1)*w] = img
    
    # 保存拼接图像
    Image.fromarray(canvas).save(save_path)
    print(f'多人脸测试图像已创建: {save_path}')
    display_image(save_path, width=400)
    
    return save_path


# ============================================================
# 创建测试图像并运行多人脸识别
# ============================================================

# 获取已注册人员名单
registered_names = list(face_db.db.keys())
print(f'已注册人员: {registered_names}')

# 创建多人脸测试图像（2个已注册 + 2个未注册）
multi_test_path = create_multi_face_image(registered_names, n_unknown=2)

# 在测试图像上运行多人脸识别
if multi_test_path:
    print('\n测试多人脸识别:')
    multi_detection_path = f'{cfg.RESULTS_DIR}/multi_face_detection_result.png'
    results = multi_recognizer.visualize(multi_test_path, save_path=multi_detection_path)

### 多人脸识别API使用示例

In [ ]:
# ============================================================
# 多人脸识别 API 使用示例
# ============================================================
# 本示例展示如何在实际应用中调用多人脸识别API

# 定义测试图像路径
demo_image = f'{cfg.RESULTS_DIR}/multi_face_test.jpg'

if os.path.exists(demo_image):
    # ==============================
    # API调用方式1：只获取识别结果
    # ==============================
    results = multi_recognizer.recognize_all(demo_image)
    
    print('=== API调用示例 ===')
    print(f'输入图像: {demo_image}')
    print(f'检测到 {len(results)} 张人脸\n')
    
    # 遍历每个检测到的人脸
    for i, res in enumerate(results, 1):
        print(f'人脸 #{i}:')
        print(f"  姓名: {res['name']}")
        print(f"  置信度: {res['confidence']:.1%}")
        print(f"  距离: {res['distance']:.3f}")
        print(f"  边界框: {res['box'].tolist()}")  # [x1, y1, x2, y2]
        print(f"  检测概率: {res['detection_prob']:.1%}")
        print('-' * 30)
    
    # ==============================
    # API调用方式2：获取结果并可视化
    # ==============================
    print('\n生成可视化结果...')
    api_demo_path = f'{cfg.RESULTS_DIR}/api_demo_result.png'
    multi_recognizer.visualize(demo_image, save_path=api_demo_path)
    
    # ==============================
    # 实际应用代码示例
    # ==============================
    """
    # 在实际项目中的使用示例：
    
    # 1. 初始化（程序启动时执行一次）
    model = FaceNet('vggface2', 128).to(device)
    face_db = FaceDatabase(model)
    face_db.load('my_database.pkl')
    recognizer = MultiFaceRecognizer(model, face_db, threshold=0.6)
    
    # 2. 处理用户上传的图片
    def handle_upload(image_path):
        results = recognizer.recognize_all(image_path)
        return [
            {
                'name': r['name'],
                'confidence': r['confidence'],
                'bbox': r['box'].tolist()
            }
            for r in results
        ]
    
    # 3. 返回JSON响应
    response = handle_upload('user_upload.jpg')
    """
    
else:
    print('请先运行前面的单元格生成测试图片')
    print(f'预期路径: {demo_image}')